In [ ]:
cd '.\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Internships\Agronne'

# Second draft

In [28]:
import modelseedpy
import cobra

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
bigg_model_path = '.\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)
modelseed = modelseedpy.biochem.from_local(modelseed_path)

def add_ms_reaction(model, rxn_id, modelseed, compartment_equivalents = {'0':'c0', '1':'e0'}, direction = 'forward'):
    ''' Add a reaction with ModelSEED parameters to the FBA simulation
    "model" (COBRA object): The metabolic model that is defined by COBRApy
    "rxn_id" (Python object, string): The ModelSEED reaction id that will be added to the model
    "Compartment_equivalents" (Python object, dictionary): The compartment codes that are used in formating the passed reactions 
    "direction" (Python object, string): The direction of the defined reaction
    "modelseed" (ModelSEED object): The ModelSEED database that describes the reaction and metabolites of the argument    
    '''
    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id)
    cobra_reaction.name = modelseed_reaction.data['name']
        
    metabolites_to_add = {}
    for metabolite, stoich in reaction_stoich.items():
        id = metabolite[0]
        compound = modelseed.get_seed_compound(id).data
        compartment_number = metabolite[1]
        compartment_string = compartment_equivalents[compartment_number]        
        
        metabolites_to_add[cobra.Metabolite(id, name = compound['name'], compartment = compartment_string)] = stoich
        
    cobra_reaction.add_metabolites(metabolites_to_add)
    cobra_reaction.reaction
    
    if direction == 'reversible':
        cobra_reaction.lower_bound = -1000
    elif direction == 'backward':
        cobra_reaction.lower_bound = -1000
        cobra_reaction.upper_bound = 0
    elif direction == 'forward':
        pass
    else:
        directions = ['forward', 'backward', 'reversible']
        print('ERROR: The \'direction\' argument is not among the accepted {}, {}, and {} values.'.format(directions[0], directions[1], directions[2]))
        direction = input('What is the direction of the reaction?')
        while direction not in directions:
            print('ERROR: The \'direction\' argument is not among the accepted {}, {}, and {} values.'.format(directions[0], directions[1], directions[2]))
            direction = input('What is the direction of the reaction?')
        add_ms_reaction(model, rxn_id, modelseed, direction = direction)
            
            
    model.add_reactions([cobra_reaction])


add_ms_reaction(model, 'rxn00002', modelseed, direction = 'backward')
print(model.reactions.get_by_id('rxn00002'))

rxn00002: cpd00001 + 3.0 cpd00067 + cpd00742 <-- 2.0 cpd00011 + 2.0 cpd00013


# First draft

In [23]:
import modelseedpy
import cobra

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
bigg_model_path = '.\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)
modelseed = modelseedpy.biochem.from_local(modelseed_path)

def add_ms_reaction(model, rxn_id, modelseed, compartment_equivalents = {'0':'c0', '1':'e0'}, direction = 'forward'):
    ''' Add a reaction with ModelSEED parameters to the FBA simulation
    "model" (COBRA object): The metabolic model that is defined by COBRApy
    "rxn_id" (Python object, string): The ModelSEED reaction id that will be added to the model
    "Compartment_equivalents" (Python object, dictionary): The compartment codes that are used in formating the passed reactions 
    "direction" (Python object, string): The direction of the defined reaction
    "modelseed" (ModelSEED object): The ModelSEED database that describes the reaction and metabolites of the argument    
    '''
    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id)
    cobra_reaction.name = modelseed_reaction.data['name']
        
    metabolites_to_add = {}
    for metabolite, stoich in reaction_stoich.items():
        id = metabolite[0]
        compound = modelseed.get_seed_compound(id).data
        compartment_number = metabolite[1]
        compartment_string = compartment_equivalents[compartment_number]        
        compound_abbreviation = '{}_{}'.format(compound['abbreviation'], compartment_string)

        metabolites_to_add[cobra.Metabolite(compound_abbreviation, name = compound['name'], compartment = compartment_string)] = stoich
        
    cobra_reaction.add_metabolites(metabolites_to_add)
    cobra_reaction.reaction
    
    if direction == 'reversible':
        cobra_reaction.lower_bound = -1000
    elif direction == 'backward':
        cobra_reaction.lower_bound = -1000
        cobra_reaction.upper_bound = 0

    model.add_reactions([cobra_reaction])


add_ms_reaction(model, 'rxn00002', modelseed, direction = 'backward')
for reaction in model.reactions:
    print(reaction)

rxn00002: allphn_c0 + h2o_c0 + 3.0 h_c0 <-- 2.0 co2_c0 + 2.0 nh4_c0


'for reaction in model.reactions:\n    print(reaction)'

### Formatting guide to the data 

In [17]:
modelseed_reaction = modelseed.get_seed_reaction('rxn00002')
modelseed_metabolite = modelseed.get_seed_compound('cpd00013')
display('reaction stoichiometry:', modelseed_reaction.cstoichiometry)
print('\n')
display('metabolite data:', modelseed_metabolite.data)
print('\n')
display('reaction data:', modelseed_reaction.data)
'''for metabolite in model.metabolites:
    print(metabolite)'''

'reaction stoichiometry:'

{('cpd00001', '0'): -1.0,
 ('cpd00067', '0'): -3.0,
 ('cpd00742', '0'): -1.0,
 ('cpd00011', '0'): 2.0,
 ('cpd00013', '0'): 2.0}

'metabolite data:'

{'id': 'cpd00013',
 'abbreviation': 'nh4',
 'name': 'NH3',
 'formula': 'H4N',
 'mass': '18.0',
 'source': 'Primary Database',
 'inchikey': 'QGZKDVFQNNGYKY-UHFFFAOYSA-O',
 'charge': 1,
 'is_core': 1,
 'is_obsolete': 0,
 'linked_compound': 'cpd19013',
 'is_cofactor': 0,
 'deltag': 19.05,
 'deltagerr': 0.26,
 'pka': nan,
 'pkb': '1:1:8.86',
 'abstract_compound': nan,
 'comprised_of': nan,
 'aliases': 'Name: Ammonia; Ammonia ion; Ammonium; Ammonium(1+); NH3; NH4+; NH4plus; ammonia; ammonium|AraCyc: AMMONIA; AMMONIUM|BiGG: nh3; nh4|BrachyCyc: AMMONIA; AMMONIUM|KEGG: C00014; C01342|MetaCyc: AMMONIA; AMMONIUM',
 'smiles': '[NH4+]'}

'reaction data:'

{'id': 'rxn00002',
 'abbreviation': 'R00005',
 'name': 'urea-1-carboxylate amidohydrolase',
 'code': '(1) cpd00001[0] + (1) cpd00742[0] <=> (2) cpd00011[0] + (2) cpd00013[0]',
 'stoichiometry': '-1:cpd00001:0:0:"H2O";-3:cpd00067:0:0:"H+";-1:cpd00742:0:0:"Allophanate";2:cpd00011:0:0:"CO2";2:cpd00013:0:0:"NH3"',
 'is_transport': 0,
 'equation': '(1) cpd00001[0] + (3) cpd00067[0] + (1) cpd00742[0] => (2) cpd00011[0] + (2) cpd00013[0]',
 'definition': '(1) H2O[0] + (3) H+[0] + (1) Allophanate[0] => (2) CO2[0] + (2) NH3[0]',
 'reversibility': '>',
 'direction': '>',
 'abstract_reaction': nan,
 'pathways': 'MetaCyc: ALLANTOINDEG-PWY (superpathway of allantoin degradation in yeast); AMINE-DEG (Amine and Polyamine Degradation); AROMATIC-COMPOUNDS-DEGRADATION (Aromatic Compound Degradation); Allantoin-degradation (Allantoin Degradation); Atrazine-Degradation (Atrazine Degradation); Degradation (Degradation/Utilization/Assimilation); PWY-5169 (cyanurate degradation); PWY-5703 (urea degradation I

'for metabolite in model.metabolites:\n    print(metabolite)'

# Brainstorming 

In [53]:
import modelseedpy
import cobra
import re
import os

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
'''if os.path.exists(modelseed_path):
    print('yes')'''


model = cobra.io.load_json_model('.\e_coli_core metabolism from BiGG.json')
metabolites = []
for metabolite in model.metabolites:
    metabolites.append(metabolite)

modelseed = modelseedpy.biochem.from_local(modelseed_path)

def add_ms_reaction(model,rxn_id,compartments,modelseed):
    ''' Add a reaction with ModelSEED parameters to the FBA simulation
    "model" (COBRA object): The metabolic model that is defined by COBRApy
    "rxn_id" (Python object, string): The ModelSEED reaction id that will be added to the model
    "compartments" (Python object, list): The biological locations of the metabolites that comprise the reaction that will be added
    "modelseed" (ModelSEED object): The ModelSEED database that describes the reaction and metabolites of the argument    
    '''
    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id)
    
    for key, value in modelseed_reaction.items():
        if key == 'name':
            cobra_reaction.name = value
        
    metabolites_to_add = {}
    for metabolite, stoich in reaction_stoich.items():
        id = metabolite[0]
        compound = modelseed.get_seed_compound(id).data
        abbreviation = compound['abbreviation']
        compartment = metabolite[1]
        
        
        if abbreviation not in metabolites:
            metabolites_to_add[cobra.Metabolite(abbreviation, name = compound['name'], compartment = compartment)] = stoich
            
            
    cobra_reaction.add_metabolites({metabolites_to_add})
    cobra_reaction.reaction
    model.add_reactions([cobra_reaction])

add_ms_reaction(model, rxn_id = 'rxn00002', modelseed)

In [ ]:
#dir(modelseedpy)
metabolites = []
for metabolite in model.metabolites:
    metabolites.append(metabolite.id)
    
rxn_id = 'rxn00002'
modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
reaction_stoich = modelseed_reaction.cstoichiometry
cobra_reaction = cobra.Reaction(rxn_id)

for key, value in modelseed_reaction.data.items():
    if key == 'name':
        cobra_reaction.name = value

metabolites_to_add = {}
for metabolite, stoich in reaction_stoich.items():
    id = metabolite[0]
    compound = modelseed.get_seed_compound(id).data
    abbreviation = compound['abbreviation']
    compartment = metabolite[1]

    
    if abbreviation not in metabolites:
        metabolites_to_add[cobra.Metabolite(abbreviation, name = compound['name'], compartment = compartment)] = stoich

#print(metabolites_to_add)

cobra_reaction.add_metabolites(metabolites_to_add)
cobra_reaction.reaction
#print(cobra_reaction)
model.add_reactions([cobra_reaction])
#dir(model.reactions)
for reaction in model.reactions:
    print(reaction)

In [ ]:
#?modelseedpy.MSGenome
metabolites = []
for metabolite in model.metabolites:
    metabolites.append(metabolite.id)
    
print(metabolites)

#dir(metabolite)
#print(metabolite.name)

In [ ]:
import cobra

model = cobra.io.load_json_model('.\e_coli_core metabolism from BiGG.json')

# generation of a sample reaction in COBRApy
reaction = cobra.Reaction('RXN01')
reaction.name = 'test name'
reaction.lower_bound = 10
reaction.upper_bound = 1000

reaction.add_metabolites({
    cobra.Metabolite('no1_c', formula = 'C2O2', name = 'metabolite 1', compartment = 'c'): 3,
    cobra.Metabolite('no2_c', formula = 'C2OH4', name = 'metabolite 2', compartment = 'c'): 2,
    cobra.Metabolite('no3_e', formula = 'C2H4', name = 'metabolite 3', compartment = 'e'): -2,
    cobra.Metabolite('no4_e', formula = 'C2H6', name = 'metabolite 4', compartment = 'e'): -3
})
reaction.reaction
#print(reaction)
#print(reaction.name)

# add the reaction to the model
model.add_reactions([reaction])

for reaction in model.reactions:
    print(reaction)

In [ ]:
import modelseedpy
import cobra
import re

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
model = cobra.io.load_json_model('.\e_coli_core metabolism from BiGG.json')
modelseed = modelseedpy.biochem.from_local(modelseed_path)
modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
cobra_reaction = cobra.Reaction(rxn_id)

for key, value in modelseed_reaction.items():
    if key == 'name':
        cobra_reaction.name = value
    elif key == 'code':
        reaction_split = reaction.split(' <=> ')
        reactants_list = reaction_split[0].split(' + ')
        products_list = reaction_split[1].split(' + ')

        reactants = []
        for element in reactants_list:
            #print(element)
            element = element.strip()
            if re.search('(\d\s|\d\/\d\s)', element):
                coefficient = re.compile('(\d\s|\d\/\d\s)')
                coefficient = coefficient.findall(element)[0]
            else:
                coefficient = ''

            element = re.sub('(\d\s|\d\/\d\s)', '', element)
            if element in nist_compounds.keys():
                reactants.append(coefficient + nist_compounds[element]['ModelSEED ID'])

            else:
                reactants.append(element)

        products = []
        for element in products_list:
            #print(element)
            element = element.strip()
            if re.search('(\d\s|\d\/\d\s)', element):
                coefficient = re.compile('(\d\s|\d\/\d\s)')
                coefficient = coefficient.findall(element)[0]
            else:
                coefficient = ''

            element = re.sub('(\d\s|\d\/\d\s)', '', element)
            if element in nist_compounds.keys():
                products.append(coefficient + nist_compounds[element]['ModelSEED ID'])

            else:
                products.append(element)

        reaction_string = value.split

        ('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string


#add_ms_reaction(model, rxn_id = 'rxn00002', , modelseed)

In [91]:
import modelseedpy
import cobra
import re
import os

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
bigg_model_path = '.\e_coli_core metabolism from BiGG.json'
'''if os.path.exists(modelseed_path):
    print('yes')'''


model = cobra.io.load_json_model(bigg_model_path)
metabolites = []
for metabolite in model.metabolites:
    metabolites.append(metabolite.id)

modelseed = modelseedpy.biochem.from_local(modelseed_path)

def add_ms_reaction(model,rxn_id,modelseed):
    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id)
    
    '''for key, value in modelseed_reaction.data.items():
        if key == 'name':
            cobra_reaction.name = value'''

    cobra_reaction.name = modelseed_reaction.data['name'] #.value

    print(cobra_reaction.name)
        
add_ms_reaction(model, 'rxn00002', modelseed)

urea-1-carboxylate amidohydrolase
